# Step by step: Passing Python functions to Claude Desktop

By the end of this notebook you'll have everything you need to run Claude with _any_ Python code you pass as a Fused User Defined Function (UDF), all running locally. 

We made this for ourselves & other data scientists who are more familiar with notebooks & Python to directly pass any Python code to Claude.

We're going to:

1. Create a simple Fused UDF to get the current time
2. Save this code locally (following the Fused UDF structure)
3. Pass it to Claude's MCP server config
4. Automatically restarting Claude with new changes


In [1]:
import fused
import json
import os
import time
from pathlib import Path

Update the following with your local file paths:

In [2]:
# This is the config file for MCP Servers that Claude Desktop uses
PATH_TO_CLAUDE_CONFIG = f"{str(Path.home())}/Library/Application Support/Claude/claude_desktop_config.json"


if not os.path.exists(PATH_TO_CLAUDE_CONFIG):
    # Creating the config file
    os.makedirs(os.path.dirname(PATH_TO_CLAUDE_CONFIG), exist_ok=True)
    with open(PATH_TO_CLAUDE_CONFIG, 'w') as f:
        json.dump({}, f)

assert os.path.exists(PATH_TO_CLAUDE_CONFIG), "Please update the PATH_TO_CLAUDE_CONFIG variable with the correct path to your Claude config file"

In [3]:
# Local path to the Claude app
CLAUDE_APP_PATH = "/Applications/Claude.app"
assert os.path.exists(CLAUDE_APP_PATH), "Please update the CLAUDE_APP_PATH variable with the correct path to your Claude app"

In [4]:
# Change this path if you're not running this from the repo root
WORKING_DIR = os.getcwd()

We can now create a simple Fused UDF that returns the current time. 

<!-- TODO: Explain a bit what a UDF (requirements, input / outputs) -->
<!-- Explain why time is good example for LLMs, because they have a cutoff period of time -->

In [5]:
@fused.udf(cache_max_age='0s')
def udf():
    """Simply return the current time in UTC"""
    import time
    import pandas as pd
    from datetime import datetime, timezone

    return pd.DataFrame({"current_utc_time": [
        datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")
    ]})

In [6]:
# We can run this UDF locally with `fused.run(udf)`
fused.run(udf)

,current_utc_time
0,2025-03-19 09:15:10


## Passing our UDFs to Claude 

The MCP server API is still changing so we're offloading some of the logic of this notebook to code we are hosting on our Public UDF repository. We'll use these utility functions to move code around & pass it to Claude's MCP server config file 

<!-- TODO: Need to explain what agents.json file looks like -->

In [7]:
commit = "5f860ab"
common = fused.load(f"https://github.com/fusedio/udfs/tree/{commit}/public/common").utils

In [8]:
# This helper function creates an intermediate file pointing to which UDF Claude can access
common.save_to_agent(
    agent_json_path = os.path.join(WORKING_DIR, "agents.json"),
    udf = udf,
    agent_name = "get_current_time",
    udf_name = "current_utc_time",
    mcp_metadata = {
        "description": "UDF that returns the current datetime in UTC",
        "parameters": "",
    }
)

In [9]:
# Checking the agents.json file
json.load(open(os.path.join(WORKING_DIR, "agents.json")))

{'agents': [{'name': 'get_current_time', 'udfs': ['current_utc_time']},
  {'name': 'fused_docs', 'udfs': ['list_public_udfs', 'reading_fused_docs']}]}

In [10]:
# Finally, we can select which Agent we want to pass to Claude in our MCP server config
common.generate_local_mcp_config(
    config_path=PATH_TO_CLAUDE_CONFIG,
    agents_list = ["get_current_time"],
    repo_path= WORKING_DIR,
)

In [11]:
# Reading claude config to confirm
json.load(open(PATH_TO_CLAUDE_CONFIG))

{'mcpServers': {'get_current_time': {'command': 'uv',
   'args': ['run',
    '--directory',
    '/Users/maximelenormand/Library/CloudStorage/Dropbox/Mac/Documents/repos/fused-mcp',
    'main.py',
    '--runtime=local',
    '--udf-names=current_utc_time',
    '--name=get_current_time']}}}

## Now restart Claude!

In [12]:
def restart_claude(claude_path: str = CLAUDE_APP_PATH):
    app_name = claude_path.split("/")[-1]

    try:
        os.system(f"pkill -f '{app_name}'")
        print(f"Killed {app_name}")
        time.sleep(2)  # Wait for shutdown
    except Exception as e:
        print(f"Claude wasn't running, so no need to kill it")

    print(f"Restarting {app_name}")
    os.system(f"open -a '{claude_path}'")  # Restart Claude

In [13]:
restart_claude()

Killed Claude.app
Restarting Claude.app


You might have to wait for a few seconds for Claude to load our MCP tools. Once tools are loaded it looks like this:

![Local Claude with MCP tools](img/Claude_setup_with_mcp_tools.png)

Everytime you run this you need to click "Allow For This Chat":

![Allow for each chat](img/claude_agree_for_chat.png)

### Change agents & Build your own!

You should now have everything you need to create your own MCP servers, save & pass them to Claude to directly interact with your code through an LLM